In [0]:
## Source to learn:
## azure: https://learn.microsoft.com/en-us/azure/databricks/machine-learning/tutorial/
## databrick: https://docs.databricks.com/machine-learning/index.html
## ref of this code: https://learn.microsoft.com/en-us/azure/databricks/_extras/notebooks/source/mlflow/ml-quickstart-training.html
## I just copy the code from ref, objective is know how to machine learning in databricks work not optimize model.
## It still need some set up and config before can run all code in ref source.

In [0]:
!pip install mlflow

     |██████▎

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 2.7 MB 37.0 MB/s 
     |████████████████████████████████| 41 kB 193 kB/s 
     |████████████████████████████████| 83 kB 1.8 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 151 kB 71.9 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 242 kB 69.7 MB/s 
     |████████████████████████████████| 96 kB 4.9 MB/s 
     |████████████████████████████████| 133 kB 67.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 610 kB 56.7 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143878 sha256=9cddec9370135aca4dc4f0485af2a3d7d67469d712757465a2f8ea5ca319206a
  Stored in directory: /root/.cache/pip/wheels/b6/90/68/94d223a35a3910c1512a8d42d9f8333ce567ef26e250a56227
Successfully built data

In [0]:
!pip install hyperopt

     |████████████████████████████████| 1.6 MB 14.7 MB/s 
     |████████████████████████████████| 2.1 MB 61.5 MB/s 
     |████████████████████████████████| 840 kB 45.8 MB/s 
     |████████████████████████████████| 200 kB 59.0 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492037 sha256=c0e31de65301e5784533c0c1073e4904744572b31e9a7e60af01ad1903dde16f
  Stored in directory: /root/.cache/pip/wheels/bf/5d/6a/2e53874f7ec4e2bede522385439531fafec8fafe005b5c3d1b
Successfully built future
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e8c0e354-ad07-457a-8870-6772bdbf30b0/bin/python -m pip install --upgrade pip' command.


In [0]:
import mlflow
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.ensemble
 
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope

# Load and preprocess data
white_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-white.csv", sep=';')
red_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-red.csv", sep=';')
white_wine['is_red'] = 0.0
red_wine['is_red'] = 1.0
data_df = pd.concat([white_wine, red_wine], axis=0)
 
# Define classification labels based on the wine quality
data_labels = data_df['quality'] >= 7
data_df = data_df.drop(['quality'], axis=1)
 
# Split 80/20 train-test
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
  data_df,
  data_labels,
  test_size=0.2,
  random_state=1
)

In [0]:
mlflow.autolog()

2023/06/07 04:15:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/06/07 04:15:09 INFO mlflow._spark_autologging: Autologging successfully enabled for spark.
2023/06/07 04:15:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2023/06/07 04:15:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


In [0]:
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
dbutils.fs.put('file:///root/.databrickscfg','[DEFAULT]\nhost=https://adb-1639995782443110.10.azuredatabricks.net\ntoken = '+token,overwrite=True)

Wrote 111 bytes.
Out[17]: True

In [0]:
with mlflow.start_run(run_name='gradient_boost') as run:
  model = sklearn.ensemble.GradientBoostingClassifier(random_state=0)
  
  # Models, parameters, and training metrics are tracked automatically
  model.fit(X_train, y_train)
 
  predicted_probs = model.predict_proba(X_test)
  roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
  
  # The AUC score on test data is not automatically logged, so log it manually
  mlflow.log_metric("test_auc", roc_auc)
  print("Test AUC of: {}".format(roc_auc))

2023/06/07 04:43:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


Test AUC of: 0.8834365701533531


In [0]:
# Start a new run and assign a run_name for future reference
with mlflow.start_run(run_name='gradient_boost') as run:
  model_2 = sklearn.ensemble.GradientBoostingClassifier(
    random_state=0, 
    
    # Try a new parameter setting for n_estimators
    n_estimators=200,
  )
  model_2.fit(X_train, y_train)
 
  predicted_probs = model_2.predict_proba(X_test)
  roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
  mlflow.log_metric("test_auc", roc_auc)
  print("Test AUC of: {}".format(roc_auc))

Test AUC of: 0.8914761673151751


In [0]:
# runmodel from another notebook
# can find the run id in mlflow experiments
#------------------------------------------
# model_loaded = mlflow.pyfunc.load_model(
#   'runs:/{run_id}/model'.format(
#     run_id=run.info.run_id
#   )
# )
 
# predictions_loaded = model_loaded.predict(X_test)
# predictions_original = model_2.predict(X_test)

Out[20]: '69224792f0d04ec9bc90ffe902bccd18'

In [0]:
# Define the search space to explore
search_space = {
  'n_estimators': scope.int(hp.quniform('n_estimators', 20, 1000, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'max_depth': scope.int(hp.quniform('max_depth', 2, 5, 1)),
}
 
def train_model(params):
  # Enable autologging on each worker
  mlflow.autolog()
  with mlflow.start_run(nested=True):
    model_hp = sklearn.ensemble.GradientBoostingClassifier(
      random_state=0,
      **params
    )
    model_hp.fit(X_train, y_train)
    predicted_probs = model_hp.predict_proba(X_test)
    # Tune based on the test AUC
    # In production settings, you could use a separate validation set instead
    roc_auc = sklearn.metrics.roc_auc_score(y_test, predicted_probs[:,1])
    mlflow.log_metric('test_auc', roc_auc)
    
    # Set the loss to -1*auc_score so fmin maximizes the auc_score
    return {'status': STATUS_OK, 'loss': -1*roc_auc}
 
# SparkTrials distributes the tuning using Spark workers
# Greater parallelism speeds processing, but each hyperparameter trial has less information from other trials
# On smaller clusters or Databricks Community Edition try setting parallelism=2
spark_trials = SparkTrials(
  parallelism=8
)
 
with mlflow.start_run(run_name='gb_hyperopt') as run:
  # Use hyperopt to find the parameters yielding the highest AUC
  best_params = fmin(
    fn=train_model, 
    space=search_space, 
    algo=tpe.suggest, 
    max_evals=32,
    trials=spark_trials)

100%|██████████| 32/32 [02:53<00:00,  5.42s/trial, best loss: -0.9166642824444954]


INFO:hyperopt-spark:Total Trials: 32: 32 succeeded, 0 failed, 0 cancelled.


In [0]:
# Sort runs by their test auc; in case of ties, use the most recent run
best_run = mlflow.search_runs(
  order_by=['metrics.test_auc DESC', 'start_time DESC'],
  max_results=10,
).iloc[0]
print('Best Run')
print('AUC: {}'.format(best_run["metrics.test_auc"]))
print('Num Estimators: {}'.format(best_run["params.n_estimators"]))
print('Max Depth: {}'.format(best_run["params.max_depth"]))
print('Learning Rate: {}'.format(best_run["params.learning_rate"]))
print('Run id: {}'.format(best_run.run_id))
 
best_model_pyfunc = mlflow.pyfunc.load_model(
  'runs:/{run_id}/model'.format(
    run_id=best_run.run_id
  )
)
best_model_predictions = best_model_pyfunc.predict(X_test[:5])
print("Test Predictions: {}".format(best_model_predictions))

Best Run
AUC: 0.9166642824444954
Num Estimators: 943
Max Depth: 5
Learning Rate: 0.3864233776522892
Run id: 484f6f23fc5e4d6f9d3f9c941a7034f0
Test Predictions: [False False False  True  True]
